In [ ]:
pip install python-dotenv

In [ ]:
import os
from os.path import join
from dotenv import load_dotenv

import pyspark as ps
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [ ]:
ps_conf = ps.SparkConf()\
            .set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol")\
            .set("mapreduce.fileoutputcommitter.marksuccessfuljobs","false")\
            .set("spark.sql.shuffle.partitions",200)
            # '_started'と'_committed_'で始まるファイルを書き込まないように設定
            # '_SUCCESS'で始まるファイルを書き込まないように設定
            # パーティション数を増やす
spark = SparkSession.builder.config(conf=ps_conf).getOrCreate()

In [ ]:
load_dotenv(join(os.getcwd(), '.env'))

In [ ]:
BASE_PATH_PYSPARK = '/mnt/adintedataexplorer'
BASE_PATH_PANDAS  = '/dbfs/mnt/adintedataexplorer'
SILVER_PATH       = '_ml-medallion/dev/test_silver/'
MITSUBISHI_CONF   = 'csv_data/mitsubishi_japan_motor_show.csv'
NAGAOKA_HNB_CONF  = 'csv_data/nagaoka_hanabi.csv'
AKA_RENGA_CONF    = 'csv_data/yokohama_aka_renga.csv'
MITSUBISHI_CSV    = 'csv_data/beacon_for_mitsubishi.csv'
NAGAOKA_HNB_CSV   = 'csv_data/beacon_for_nagaoka.csv'
AKA_RENGA_CSV     = 'csv_data/beacon_for_aka_renga.csv'
SPECIFIED_COMB    = (MITSUBISHI_CONF, MITSUBISHI_CSV)

In [ ]:
path = BASE_PATH_PYSPARK + SILVER_PATH + SPECIFIED_COMB[0]
df_csv = spark.read\
                .option('inferSchema', 'True')\
                .option('header', 'True')\
                .csv(path)

df_csv.display()
utid_list = sorted(df_csv.select("unit_id").drop_duplicates().rdd.flatMap(lambda x: x).collect())

In [ ]:
START_DAY = '2023-10-20'
END_DAY   = '2023-11-16'
AIBEACON_PATH = 'adinte.aibeacon_wifi_log'

df_raw_data = spark.table(AIBEACON_PATH)\
                    .withColumn('date',     F.to_date(col('date')))\
                    .withColumn('datetime', F.to_timestamp(col('datetime')))\
                    .filter((col('date') >= START_DAY) & (col('date') <= END_DAY))\
                    .filter(col('randomized') == '1')\
                    .filter(col('unit_id').isin(utid_list))
df_raw_data.display()

In [ ]:
path = BASE_PATH_PANDAS + SILVER_PATH + SPECIFIED_COMB[1]
pd_raw_data = df_raw_data.toPandas()\
                .to_csv(path, index=False, header=True)